### Ladi_ai_RAG

- API DOCS : http://43.202.173.71:3000/
- TASK : 루틴 추천
- 루틴 관련 데이터 저장(vectorDB) -> 검색(Retrieve) -> LLM(gpt-4o)의 Context로 사용

In [2]:
import requests
import json

# API 엔드포인트 URL
api_url = "http://43.202.173.71:3000/api/endpoint"  # 실제 엔드포인트 URL로 변경해야 합니다

# 요청 데이터
request_data = {
    "Difficult": "의욕 떨어짐, 목표 너무 큼",
    "Job": "학생",
    "gender": "남성",
    "age": 24,
    "goal": "꾸준히 운동하기"
}

# POST 요청 보내기
response = requests.post(api_url, json=request_data)

# 응답 처리
if response.status_code == 200:
    # 성공적인 응답
    response_data = response.json()
    print("API 응답:")
    print(json.dumps(response_data, indent=2, ensure_ascii=False))
else:
    # 오류 응답
    print(f"오류 발생: {response.status_code}")
    print(response.text)

오류 발생: 404
<!DOCTYPE html><html><head><title></title><link rel="stylesheet" href="/stylesheets/style.css"></head><body><h1>Not Found</h1><h2>404</h2><pre>NotFoundError: Not Found
    at file:///home/ubuntu/life-designer-server/app.js:39:8
    at Layer.handle [as handle_request] (/home/ubuntu/life-designer-server/node_modules/express/lib/router/layer.js:95:5)
    at trim_prefix (/home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:317:13)
    at /home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:284:7
    at Function.process_params (/home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:335:12)
    at next (/home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:275:10)
    at /home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:635:15
    at next (/home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:260:14)
    at Function.handle (/home/ubuntu/life-designer-s

In [3]:
import requests

# API 엔드포인트 URL
token_url = "http://43.202.173.71:3000/auth/token"

# 요청 데이터 (필요한 경우 수정)
request_data = {
    "username": "your_username",  # 사용자 이름
    "password": "your_password"   # 비밀번호
}

# POST 요청 보내기
response = requests.post(token_url)

# 응답 처리
if response.status_code == 200:
    # 성공적인 응답
    token_data = response.json()
    print("토큰 응답:")
    print(json.dumps(token_data, indent=2, ensure_ascii=False))
else:
    # 오류 응답
    print(f"오류 발생: {response.status_code}")
    print(response.text)


오류 발생: 401
Unauthorized


In [4]:
import requests
# response=requests.get('http://43.202.173.71:3000/statistics')
# print(response.json())

response = requests.get('http://43.202.173.71:3000/statistics')
if response.status_code == 200:
    try:
        data = response.json()
    except ValueError:
        print("응답이 JSON 형식이 아닙니다:", response.text)
else:
    print(f"오류 발생: {response.status_code} - {response.text}")


오류 발생: 401 - Unauthorized


In [5]:
from flask import Flask, jsonify
import requests

app = Flask(__name__)

@app.route('/get_statistics', methods=['GET'])
def get_statistics():
    try:
        # localhost:3000/statistics에 GET 요청 보내기
        response = requests.get('http://43.202.173.71:3000/statistics')
        
        # 응답 상태 코드 확인
        if response.status_code == 200:
            # 성공적으로 데이터를 받아왔을 때
            data = response.json()
            return jsonify(data), 200
        else:
            # 오류 발생 시
            return jsonify({"error": "외부 API에서 데이터를 가져오는 데 실패했습니다."}), response.status_code
    
    except requests.RequestException as e:
        # 네트워크 오류 등 예외 처리
        return jsonify({"error": f"요청 중 오류 발생: {str(e)}"}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/ipykernel/kernelapp.py", l

SystemExit: 1

/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
import json
import os

def load_json_files(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                data.extend(json.load(file))
    return data

json_data = load_json_files('data/')

In [26]:
import json
from langchain.schema import Document
from pprint import pprint

# JSON 파일 로드 함수 정의
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# JSON 데이터 로드
data = load_json_file("data/dummy.json")

# 필요한 정보 추출 및 Document 객체 생성
docs = [
    Document(
        page_content=item["text"],
        metadata={
            "id": item["id"],
            "emoji": item["metadata"].get("emoji"),
            "miniuteDuration": item["metadata"].get("miniuteDuration")
        }
    )
    for item in data
]

# 결과 출력
pprint(docs)

[Document(metadata={'id': '1', 'emoji': '🏃', 'miniuteDuration': 5}, page_content='가볍게 5분 걷기'),
 Document(metadata={'id': '2', 'emoji': '🧘', 'miniuteDuration': 5}, page_content='스트레칭으로 몸 풀기'),
 Document(metadata={'id': '3', 'emoji': '💪', 'miniuteDuration': 3}, page_content='팔굽혀펴기 10회'),
 Document(metadata={'id': '4', 'emoji': '🦵', 'miniuteDuration': 5}, page_content='스쿼트 15회'),
 Document(metadata={'id': '5', 'emoji': '🚶', 'miniuteDuration': 10}, page_content='10분 걷기로 마무리'),
 Document(metadata={'id': '6', 'emoji': '🧘\u200d♂️', 'miniuteDuration': 15}, page_content='요가로 몸과 마음 안정시키기'),
 Document(metadata={'id': '7', 'emoji': '🚴', 'miniuteDuration': 20}, page_content='자전거 타기 20분'),
 Document(metadata={'id': '8', 'emoji': '🤸', 'miniuteDuration': 2}, page_content='플랭크 1분 유지'),
 Document(metadata={'id': '9', 'emoji': '🦵', 'miniuteDuration': 5}, page_content='런지 10회'),
 Document(metadata={'id': '10', 'emoji': '🏃\u200d♂️', 'miniuteDuration': 15}, page_content='가벼운 조깅 15분')]


In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 1. 벡터 저장소 생성
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query")
vectorstore = Chroma(embedding_function=passage_embeddings)


/var/folders/ng/3fkn67lx65g3fmz9qjw7gdp80000gn/T/ipykernel_88821/777457567.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(embedding_function=passage_embeddings)


In [8]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

fs = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    passage_embeddings, fs, namespace=passage_embeddings.model
)

# 2. 문서 임베딩 및 저장
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
vectorstore.add_documents(texts)


NameError: name 'docs' is not defined

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 3. 검색 기능 구현
retriever = vectorstore.as_retriever()

# 4. LLM 설정 (OpenAI API 키가 필요합니다)
model = ChatOpenAI(temperature=0,
                  model_name="gpt-4o-mini-2024-07-18",
                  function_map={
                      "subRoutine": {
                          "description": "운동 루틴을 추천합니다.",
                          "parameters": {
                              "type": "object",
                              "properties": {
                                  "emoji": {"type": "string"},
                                  "routineDetail": {"type": "string"},
                                  "miniuteDuration": {"type": "string"}
                              }
                          }
                      }
                  })
# 더미 데이터 생성
dummy_data = {
    "context": "운동 루틴에 대한 정보가 포함된 문서 내용입니다.",
    "question": "운동 루틴에 대해 설명해주세요.",
    "age": 24,
    "gender": "남성",
    "Job": "학생",
    "Difficult": "의욕 떨어짐, 목표 너무 큼",
    "goal": "꾸준히 운동하기"
}

# 5. RAG 파이프라인 구축
template = """당신은 API 서버입니다.
{context}
사용자의 나이는 {age}입니다.
사용자의 성별은 {gender}입니다.
사용자의 직업은 {Job}입니다.
사용자의 겪고 있는 어려움은 {Difficult}입니다.
사용자의 목표는 {goal}입니다.

질문: {question}
답변:
응답 형식
{{
  "subRoutine": 
    }}
      "emoji": "이모지",
      "routineDetail": "루틴 설명",
      "miniuteDuration": "시간(분 단위)"
    }}
  
}}"""


prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question", "age", "gender", "Job", "Difficult", "goal"]
)

# RAG 체인 생성
chain = (
     prompt
    | model
    | StrOutputParser()
)

# 결과 생성
result = chain.invoke(dummy_data,)

# 결과 출력
print(result)

/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: WARNING! function_map is not default parameter.
                function_map was transferred to model_kwargs.
                Please confirm that function_map is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


KeyError: 'Input to PromptTemplate is missing variables {\'\\n  "subRoutine"\'}.  Expected: [\'\\n  "subRoutine"\', \'Difficult\', \'Job\', \'age\', \'context\', \'gender\', \'goal\', \'question\'] Received: [\'context\', \'question\', \'age\', \'gender\', \'Job\', \'Difficult\', \'goal\']\nNote: if you intended {\n  "subRoutine"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n  "subRoutine"}}\'.'